In [1]:
import networkx as nx
import pandas as pd
import dill
from tqdm import tqdm
import graph_tool.all as gt
from tqdm import tqdm
import copy
from typing import Generic, TypeVar, cast
from typing import List, Tuple
from collections import defaultdict
import random, json
import networkx as nx
import datetime, os
from multiprocess import Pool

In [2]:
# loading the networkx graph
nxG = None
print("loading graph")
with open('graph.pkl', 'rb') as f:
    nxG = dill.load(f)
print("graph loaded")
print(nxG)

loading graph
graph loaded
MultiDiGraph with 789856 nodes and 1776743 edges


In [3]:
# here's a method that keeps nodes from a list
# and it should also remove edges between nodes of a certain kind, if provided
def filter_graph(G: nx.Graph, node_labels_to_keep: list=[], edges_to_keep: List[Tuple]=[]) -> nx.Graph:
    '''
    takes in a graph G, a list of node labels to keep i.e. ["User", "Device"]
    and a list of tuples (order doesn't matter) saying between which kinds of nodes
    we want to keep edges, i.e. [("User", "Device")]
    '''
    print("graph before filtering: ", G)

    # first find all nodes to remove
    if len(node_labels_to_keep) > 0:
        nodes_to_remove = []
        attrs = nx.get_node_attributes(G, 'labels') # this makes it faster
        for n in tqdm(G.nodes()):
            # print(list(nx.get_node_attributes(G, 'labels')[n])[0])
            if list(attrs[n])[0] not in node_labels_to_keep:
                nodes_to_remove.append(n)
        
        # then remove them
        Gnew = copy.deepcopy(G)
        Gnew.remove_nodes_from(nodes_to_remove)

    # then find all edges to remove
    if len(edges_to_keep) > 0:
        edges_to_delete = []
        attrsnew = nx.get_node_attributes(Gnew, 'labels')
        for u, v, attr in Gnew.edges(data=True):
            ntu = list(attrsnew[u])[0]
            ntv = list(attrsnew[v])[0]
            keep = False
            for edge_tuple in edges_to_keep:
                if str((ntu, ntv)) == edge_tuple or str((ntv, ntu)) == edge_tuple:
                    keep = True
            if not keep:
                edges_to_delete.append((u, v))

        # then remove them
        Gnew.remove_edges_from(edges_to_delete)

    print("graph after filtering:", Gnew)

    return Gnew

In [10]:
# defining graph_tool methods

def convert_networkx_to_graphtool(nx_graph):
    # print("part_1")
    gt_graph = gt.Graph(directed=nx_graph.is_directed())
    # print("part_2")
    gt_graph.add_vertex(len(nx_graph))

    # Create a mapping between original labels and new integer labels
    # print("part_3")
    label_to_index = {label: index for index, label in enumerate(nx_graph.nodes)}
    index_to_label = {index: label for index, label in enumerate(nx_graph.nodes)}

    print("adding edges")
    for edge in tqdm(nx_graph.edges):
        gt_graph.add_edge(label_to_index[edge[0]], label_to_index[edge[1]])

    return gt_graph, label_to_index, index_to_label

def compute_betweenness(g):
    vb, eb = gt.betweenness(g)
    betweenness = {}
    for v in g.vertices():
        betweenness[int(v)] = vb[v]
    return betweenness

def compute_eigenvector_centrality(g):
    _, ev = gt.eigenvector(g)
    eigenvector_centrality = {}
    for v in g.vertices():
        eigenvector_centrality[int(v)] = ev[v]
    return eigenvector_centrality

def compute_closeness(g):
    closeness = gt.closeness(g)
    closeness_dict = {}
    for v in g.vertices():
        closeness_dict[int(v)] = closeness[v]
    return closeness_dict

def compute_degree(g):
    degree_dict = {}
    for v in g.vertices():
        degree_dict[int(v)] = v.out_degree()
    return degree_dict

def compute_SIS_infection_prob(g, beta, mu):
    N = g.num_vertices()
    infection_prob = [0] * N
    for i in range(N):
        infection_prob[i] = 1 - (1 - beta) ** (mu * g.vertex(i).out_degree())
    return infection_prob

def compute_page_rank(g, damping=0.85, epsilon=1e-6):
    pr = gt.pagerank(g, damping=damping, epsilon=epsilon)
    page_rank = {}
    for v in g.vertices():
        page_rank[int(v)] = pr[v]
    return page_rank

    
# def SI_infection_simulation(g, n, t, label_probabilities=None):
#     N = g.num_vertices()
#     infected_count = [0] * t

#     def infect_node(start_node, timesteps):
#         infected = defaultdict(bool)
#         infected[start_node] = True
#         new_infected = [start_node]
#         count = 1

#         for t in range(timesteps):
#             if count >= n:
#                 break
#             next_infected = []

#             for node in new_infected:
#                 neighbors = [v for v in node.out_neighbors()]
#                 for neighbor in neighbors:
#                     if not infected[int(neighbor)]:
#                         if label_probabilities:
#                             infector_label = g.vp.label[node]
#                             infected_label = g.vp.label[neighbor]
#                             infection_prob = label_probabilities[(infector_label, infected_label)]
#                         else:
#                             infection_prob = 1

#                         if random() < infection_prob:
#                             infected[int(neighbor)] = True
#                             next_infected.append(neighbor)
#                             count += 1
#                             if count >= n:
#                                 break

#             new_infected = next_infected
#             infected_count[t] += count

#     for node in tqdm(g.vertices()):
#         infect_node(node, t)

#     return [count / N for count in infected_count]

# def SI_infection_simulation_timestamps(g, n, t, nx_graph, label_to_index, index_to_label, label_probabilities=None):
#     nx_nodeattrs = nx.get_node_attributes(nx_graph, 'labels')
#     nx_edgeattrs = nx.get_edge_attributes(nx_graph, 'type')
#     def get_timestamp_from_edge(n1, n2):
        

#     N = g.num_vertices()
#     infected_count = [0] * t

#     def infect_node(start_node, timesteps):
#         infected = defaultdict(bool)
#         infected[start_node] = True
#         new_infected = [start_node]
#         count = 1

#         for current_t in range(timesteps):
#             if count >= n:
#                 break
#             next_infected = []

#             for node in new_infected:
#                 neighbors = [(v, g.ep.timestamps[e]) for e, v in zip(node.out_edges(), node.out_neighbors())]

#                 for neighbor, timestamps in neighbors:
#                     if not infected[int(neighbor)] and current_t in timestamps:
#                         if label_probabilities:
#                             infector_label = g.vp.label[node]
#                             infected_label = g.vp.label[neighbor]
#                             infection_prob = label_probabilities[(infector_label, infected_label)]
#                         else:
#                             infection_prob = 1

#                         if random.random() < infection_prob:
#                             infected[int(neighbor)] = True
#                             next_infected.append(neighbor)
#                             count += 1
#                             if count >= n:
#                                 break

#             new_infected = next_infected
#             infected_count[current_t] += count

#     for node in g.vertices():
#         infect_node(node, t)

#     return [count / N for count in infected_count]

def get_edge_time(edge_data):
    if 'transactionDateTime' in edge_data:
        return edge_data['transactionDateTime']
    elif 'cardDate' in edge_data:
        return datetime.datetime.combine(edge_data['cardDate'], datetime.time())
    elif 'deviceDate' in edge_data:
        return datetime.datetime.combine(edge_data['deviceDate'], datetime.time())
    elif 'ipDate' in edge_data:
        return datetime.datetime.combine(edge_data['ipDate'], datetime.time())
    else:
        return None

def _si_infection_step(args):
    graph, node, t, label_prob = args
    infected_nodes = set([node])
    new_infected_nodes = set([node])

    for _ in range(t):
        next_new_infected_nodes = set()
        for infected_node in new_infected_nodes:
            neighbors = list(graph.neighbors(infected_node))
            for neighbor in neighbors:
                if neighbor not in infected_nodes and random.random() <= label_prob[list(graph.nodes[infected_node]['labels'])[0]][list(graph.nodes[neighbor]['labels'])[0]]:
                    next_new_infected_nodes.add(neighbor)
                    infected_nodes.add(neighbor)
        new_infected_nodes = next_new_infected_nodes
    return len(infected_nodes)

def si_infection_simulation(graph, n=100, t=100, label_prob=None):
    if label_prob is None:
        label_prob = {
            # label1: {label2: 1.0 for label2 in graph.graph['Label']} for label1 in graph.graph['Label']
            "User": {
                "User": 1,
                "Device": 1,
                "Card": 1,
                "IP": 1,
            },
            "Device": {
                "User": 1,
                "Device": 1,
                "Card": 1,
                "IP": 1,
            },
            "Card": {
                "User": 1,
                "Device": 1,
                "Card": 1,
                "IP": 1,
            },
            "IP": {
                "User": 1,
                "Device": 1,
                "Card": 1,
                "IP": 1,
            },
        }

    nodes = list(graph.nodes)
    args = [(graph, node, t, label_prob) for node in nodes]

    with Pool() as pool:
        infection_counts = list(tqdm(pool.imap_unordered(_si_infection_step, args), total=len(args), desc="Infection simulation"))

    return infection_counts

# Example usage:
# G = ... # Load the graph
# n = 5
# t = 10
# label_prob = { ... } # Optional custom label probabilities
# infection_counts = si_infection_simulation(G, n, t, label_prob)
# print(infection_counts)

# Example usage:
# label_probabilities = {("A", "A"): 0.9, ("A", "B"): 0.5, ("B", "A"): 0.5, ("B", "B"): 0.9}
# infected_counts = SI_infection_simulation_timestamps(g, 10, 100, label_probabilities)

In [11]:
# computing metrics
def compute_metrics(G: gt.Graph, index_to_label: dict):

    g = G

    beta = 0.5
    mu = 0.3

    print("computing closeness")
    closeness = compute_closeness(g)

    print("computing degree")
    degree = compute_degree(g)

    print("computing betweenness")
    betweenness = compute_betweenness(g)

    print("computing eigenvector_centrality")
    eigenvector_centrality = compute_eigenvector_centrality(g)

    print("computing SIS_infection_prob")
    SIS_infection_prob = compute_SIS_infection_prob(g, beta, mu)

    print("computing pagerank")
    page_rank = compute_page_rank(g)

    print("computation done")

    # print(f"Node : Betweenness : Eigenvector Centrality : Closeness : Degree : SIS Infection Probability : Page Rank")
    # for node in list(g.vertices())[:10]:
    #     print(f"{int(node)} : {betweenness[int(node)]} : {eigenvector_centrality[int(node)]} : {closeness[int(node)]} : {degree[int(node)]} : {SIS_infection_prob[int(node)]} : {page_rank[int(node)]}")
    #     # print(f"Node {int(node)}: Betweenness: {betweenness[int(node)]}, Eigenvector Centrality: {eigenvector_centrality[int(node)]}, Closeness: {closeness[int(node)]}, Degree: {degree[int(node)]}, SIS Infection Probability: {SIS_infection_prob[int(node)]}")
    
    data_dict = {}
    for node in g.vertices():
        data_dict[index_to_label[int(node)]] = dict(
            closeness=closeness[int(node)],
            degree=degree[int(node)],
            betweenness=betweenness[int(node)],
            eigenvector_centrality=eigenvector_centrality[int(node)],
            sis_infection_prob=SIS_infection_prob[int(node)],
            page_rank=page_rank[int(node)],
        )
    return data_dict

In [12]:
def save_graphs(nx_graph, gt_graph, graph_name, label_to_index, index_to_label):
    if not os.path.exists('graphs'):
        os.makedirs('graphs')

    nx_file = os.path.join('graphs', f'{graph_name}_networkx.dill')
    gt_file = os.path.join('graphs', f'{graph_name}_graphtool.dill')

    if not os.path.exists(nx_file):
        with open(nx_file, 'wb') as f:
            dill.dump(nx_graph, f)

    if not os.path.exists(gt_file):
        with open(gt_file, 'wb') as f:
            dill.dump((gt_graph, label_to_index, index_to_label), f)

def load_graphs(graph_name):
    nx_file = os.path.join('graphs', f'{graph_name}_networkx.dill')
    gt_file = os.path.join('graphs', f'{graph_name}_graphtool.dill')

    if os.path.exists(nx_file) and os.path.exists(gt_file):
        with open(nx_file, 'rb') as f:
            nx_graph = dill.load(f)

        with open(gt_file, 'rb') as f:
            gt_graph, label_to_index, index_to_label = dill.load(f)

        return nx_graph, gt_graph, label_to_index, index_to_label
    else:
        return None, None, None, None

In [14]:
# the experiment then goes like this:
# 1. instantiate the graphs you're interested in
# 2. run analysis on each, only keep the data_dict
# 3. put the dicts together in a dict with simple names

graphs_we_want = {
    "only_users": dict(
        node_labels_to_keep=['User'],
        edges_to_keep=[],
    ),
    # "users_and_cards": dict(
    #     node_labels_to_keep=['User', 'Card'],
    #     edges_to_keep=[],
    # ),
    # "users_and_devices": dict(
    #     node_labels_to_keep=['User', 'Device'],
    #     edges_to_keep=[],
    # ),
    # "users_and_ips": dict(
    #     node_labels_to_keep=['User', 'IP'],
    #     edges_to_keep=[],
    # ),
    # "users_and_cards_without_user_to_user_edges": dict(
    #     node_labels_to_keep=['User', 'Card'],
    #     edges_to_keep=[('User', 'Card')],
    # ),
}

results = {}
for graph_name, graph_info in graphs_we_want.items():
    print("\n\ncomputing graph ", graph_name)
    # 1. instantiate the graphs you're interested in
    nx_graph, gt_graph, label_to_index, index_to_label = load_graphs(graph_name)
    if nx_graph is None or gt_graph is None:
        nx_graph = filter_graph(nxG, graph_info['node_labels_to_keep'], graph_info['edges_to_keep'])
        gt_graph, label_to_index, index_to_label = convert_networkx_to_graphtool(nx_graph)
        save_graphs(nx_graph, gt_graph, graph_name, label_to_index, index_to_label)

    # 2. run analysis on each, only keep the data_dict
    result = compute_metrics(gt_graph, index_to_label)
    
    # 3. put the dicts together in a dict with simple names
    # results[graph_name] = result
    results = si_infection_simulation(nx_graph)
    print(results)
# nx_graph = filter_graph(nxG, graphs_we_want["users_and_cards"]['node_labels_to_keep'], graphs_we_want["users_and_cards"]['edges_to_keep'])
# gt_graph, label_to_index, index_to_label = convert_networkx_to_graphtool(nx_graph)



computing graph  only_users
graph before filtering:  MultiDiGraph with 789856 nodes and 1776743 edges


100%|██████████| 789856/789856 [00:00<00:00, 2376665.75it/s]


In [ ]:
# print(list(gt_graph.vertices())[:10])
# vertex = list(gt_graph.vertices())[154]
# print(dir(vertex))
# vertex object attributes
# 'all_edges', 'all_neighbors', 'all_neighbours', 'graph_ptr', 'graph_type',
# 'in_degree', 'in_edges', 'in_neighbors', 'in_neighbours', 'is_valid',
# 'out_degree', 'out_edges', 'out_neighbors', 'out_neighbours'

# for v in gt_graph.vertices():
#     v['name'] = "brian"
#     for prop in v.vp:
#         print(prop)

# edat = nx.get_edge_attributes(nx_graph, 'properties')
# # for k, v in list(edat.items())[:10]:
# #     print(k, v)
# for u, v, keys, attrs in list(nx_graph.edges(data=True, keys=True))[:10]:
#     print(attrs)
#     # print(u, v)
#     print(edat[(u, v, keys)])

In [ ]:
# with open("compute_metrics_results.json", 'w') as f:
#     json.dump(results, f, indent=4)